In [1]:
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
from datetime import datetime
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

Duplicate key in file '/Users/hwesungryu/.matplotlib/matplotlibrc' line #2.
Duplicate key in file '/Users/hwesungryu/.matplotlib/matplotlibrc' line #3.


In [2]:
online_action = pd.read_csv('./data/제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv', engine='python', encoding='utf-8-sig')
transaction = pd.read_csv('./data/제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv', engine='python', encoding='utf-8-sig')
customer = pd.read_csv('./data/제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv', engine='python', encoding='utf-8-sig')
production = pd.read_csv('./m_data/제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv', engine='python', encoding='utf-8-sig')

## 상품 소분류에 따라 대분류 묶기

In [3]:
production

,pd_c,clac_nm0,clac_nm1,clac_nm2,clac_nm3
0,1,"Sports, Leisure",Automotive Products,Automotive Replacement Repair / Maintanance Kits,Automobile Oil / Additives
1,2,"Sports, Leisure",Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Lights
2,3,"Sports, Leisure",Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Paint
3,4,"Sports, Leisure",Automotive Products,Automotive Replacement Repair / Maintanance Kits,Filters
4,5,"Sports, Leisure",Automotive Products,Automotive Replacement Repair / Maintanance Kits,Wiper Blades
...,...,...,...,...,...
1662,1663,Fashion_Clothes,Women's Clothing,Women's Upper Bodywear / Tops,Women's Cardigans
1663,1664,Fashion_Clothes,Women's Clothing,Women's Upper Bodywear / Tops,Women's Sweaters / Pullover
1664,1665,Fashion_Clothes,Women's Clothing,Women's Upper Bodywear / Tops,Women's Sweatshirts / Hoodies
1665,1666,Fashion_Clothes,Women's Clothing,Women's Upper Bodywear / Tops,Women's T-shirts / Tops


In [4]:
clac_nm0=production[['pd_c','clac_nm0']]

In [5]:
transaction.pd_c.replace('unknown', 9999, inplace=True)

In [6]:
transaction.pd_c=transaction.pd_c.apply(lambda x: int(x))

In [7]:
clac_nm=transaction.merge(clac_nm0, on='pd_c')

In [8]:
# clac_nm.to_csv('clac_nm.csv')

In [9]:
clac_nm.clac_nm0=clac_nm.clac_nm0.apply(lambda x: str(x))

In [10]:
clac_nm.clnt_id=clac_nm.clnt_id.apply(lambda x: str(x))

## 구매했던 품목 종류 갯수 비율

In [11]:
# 상품군별 포함되는 소분류 갯수 Dictionary 만들기
clac_nm0_dict=dict(production.clac_nm0.value_counts())

In [12]:
# 클라이언트 Id별 구매한 상품의 대분류 갯수 Table
clac_nm0_count=clac_nm.pivot_table(index='clnt_id', columns='clac_nm0', aggfunc='count', values=['pd_c'], fill_value=0)

In [13]:
clac_nm0_count.columns.names = ['id_x1', 'clac_nm0']

In [14]:
clac_nm0_count=clac_nm0_count.stack(level=0).reset_index('id_x1').drop('id_x1', axis=1)

In [15]:
clac_nm0_count=clac_nm0_count.astype(float)

In [16]:
# 대분류별로 포함되는 상품 갯수가 다르므로 비율로 변경

for i in range(0,len(clac_nm0_count)):
    for j in range(0,len(clac_nm0_count.columns)):
        clac_nm0_count.iloc[i][j]=clac_nm0_count.iloc[i][j]/clac_nm0_dict[clac_nm0_count.columns[j]]

In [17]:
clac_nm0_count

clac_nm0,Baby,Beauty,"Digiter, Appliences",Fashion_Accessories,Fashion_Clothes,Foods,Interier,"Living, Health",Others,"Sports, Leisure","Travel, Culture"
clnt_id,,,,,,,,,,,
10002,0.269231,0.046512,0.0,0.042553,0.000000,0.055901,0.000000,0.007092,0.000000,0.00,0.12
10009,0.000000,0.000000,0.0,0.007092,0.005682,0.041925,0.000000,0.003546,0.000000,0.00,0.00
10030,0.000000,0.034884,0.0,0.014184,0.005682,0.041925,0.000000,0.010638,0.000000,0.00,0.00
10040,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.00,0.00
10046,0.000000,0.011628,0.0,0.000000,0.000000,0.032609,0.000000,0.007092,0.000000,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
9956,0.000000,0.000000,0.0,0.007092,0.022727,0.003106,0.000000,0.024823,0.333333,0.00,0.00
9975,0.000000,0.000000,0.0,0.007092,0.005682,0.000000,0.000000,0.000000,0.000000,0.01,0.00
9980,0.000000,0.000000,0.0,0.000000,0.000000,0.027950,0.000000,0.003546,0.000000,0.00,0.08


## 총페이지 조회건수, 세션에 머무는 시간

In [18]:
a=online_action.pivot_table(index='clnt_id', values=['tot_pag_view_ct', 'tot_sess_hr_v'], aggfunc='max')

In [19]:
b=online_action.pivot_table(index='clnt_id', values=['tot_pag_view_ct', 'tot_sess_hr_v'], aggfunc='min')

In [20]:
c=online_action.pivot_table(index='clnt_id', values=['tot_pag_view_ct', 'tot_sess_hr_v'], aggfunc='mean')

In [21]:
d=a-b

In [22]:
d.fillna(1, inplace=True)

In [23]:
d.replace(to_replace=0, value=1, inplace=True)

In [24]:
e=c/d

In [25]:
e

,tot_pag_view_ct,tot_sess_hr_v
clnt_id,,
1,1.151613,1.113333
2,0.532443,0.459059
3,89.000000,1388.000000
4,115.000000,950.000000
5,37.000000,879.000000
...,...,...
72425,0.937804,0.913471
72426,6.000000,334.000000
72427,0.860140,0.952775


## Back_nums

In [26]:
Back=online_action[['clnt_id', 'sess_id', 'hit_seq', 'action_type']]

In [27]:
Back=Back.sort_values(by=['clnt_id','sess_id', 'hit_seq'])

In [28]:
Back1=Back.set_index(['clnt_id', 'sess_id', 'hit_seq'])

In [ ]:
Back_num_list_sess = []
Back_num_list = []
Back_num=0

for i in list(Back.clnt_id.unique()):
    for j in range(1, max(Back1.loc[i].reset_index('sess_id').sess_id)+1):
        for k in range(1, max(Back1.loc[i].loc[j].reset_index('hit_seq').hit_seq)):
            if Back1.loc[i].loc[j].loc[k].iloc[0] - Back1.loc[i].loc[j].loc[k+1].iloc[0] > 0:
                Back_num += 1
            else:
                continue
        Back_num_list_sess.append(Back_num)
        Back_num=0
    
    Back_num_list.append(sum(Back_num_list_sess)/len(Back_num_list_sess))
    Back_num_list_sess = []

    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278

In [ ]:
a['Back_nums'] = 0

In [ ]:
a['Back_nums'] = Back_num_list

In [ ]:
a.to_csv('배성.csv')